# Part III. Tools for Causal Discovery – CaDis 2023

In [ ]:
# We make sure that we have the necessary packages
install.packages("BiocManager")

BiocManager::install("Rgraphviz")
BiocManager::install("graph")
BiocManager::install("RBGL")

install.packages("fastICA",dependencies = TRUE)
install.packages("pcalg",dependencies = TRUE)

In [ ]:
# then we load them
library(pcalg)
library(graph)
library(Rgraphviz)
library(RBGL)
set.seed(124)

### Exploring elementary objects

In [ ]:
# Loading a data set with binary variables
data(gmB)

# The gmB was loaded in the workspace and contains two elements

# data (200 observations)
gmB$x[0:20,]

# model represeted by a DAG
gmB$g
# list of nodes and edges
gmB$g@nodes
gmB$g@edgeL

# plotting the DAG
plot(gmB$g)

In [ ]:
# more data sets
# [B]inary, [D]iscrete, [G]aussian
par(mfrow=c(2,2))
data(gmD)
plot(gmD$g)

data(gmG)
plot(gmG$g)

# visualizing gmG distribution of some variables
hist(gmG$x[,1])
hist(gmG$x[,2])

In [ ]:
# creating a DAG
A <- rbind(c(0,1,0,0,1),
           c(0,0,0,1,1),
           c(1,0,0,1,0),
           c(1,0,0,0,1),
           c(0,0,0,0,0))

gA <- getGraph(A)
par(mfrow=c(1,2))
plot(gA,main="Graph from AM")

## Working with simulated data
# generating a random DAG
myDAG1 <- randomDAG(n = 6, prob= 0.5, lB = 0.1, uB = 1)
myDAG2 <- randDAG(6,3)
par(mfrow=c(1,2))
plot(myDAG1,main="randomDAG")
plot(myDAG2,main="randDAG")


In [ ]:
# listing edges
myDAG1@edgeL
myDAG2@edgeL

# myDAG2 is not topologically ordering, then we can use 'tsort()'
tsort(myDAG2)

In [ ]:
# For now, we will use a DAG generate via randomDAG (topol. ordered)
par(mfrow=c(1,2))

p <- 8
pconn <- 0.25
myDAG1 <- randomDAG(p, prob = pconn, lB=0.1, uB=1)

plot(myDAG1, main = "randomDAG")


### Estimate (Initial) Skeleton of a DAG

In [ ]:
## Using Gaussian data
data(gmG)
n <- nrow (gmG8$x)
V <- colnames(gmG8$x) # labels aka node names
# estimate Skeleton
print(V)
skel.fit <- skeleton(suffStat = list(C = cor(gmG8$x), n = n),
                     indepTest = gaussCItest, ## (partial correlations)
                     alpha = 0.01, labels = V, verbose = TRUE)

# show estimated Skeleton
par(mfrow=c(1,2))
plot(skel.fit, main = "Estimated Skeleton")
plot(gmG8$g, main = "True DAG")


In [ ]:
## Using discrete data
data(gmD)
V <- colnames(gmD$x) # labels aka node names
## define sufficient statistics
suffStat <- list(dm = gmD$x, nlev = c(3,2,3,4,2), adaptDF = FALSE)
## estimate Skeleton
skel.fit <- skeleton(suffStat,
                     indepTest = disCItest, ## (G^2 statistics independence test)
                     alpha = 0.01, labels = V, verbose = FALSE)
## show estimated Skeleton
par(mfrow = c(1,2))
plot(skel.fit, main = "Estimated Skeleton")
plot(gmD$g, main = "True DAG")


In [ ]:
## Using binary data
data(gmB)
X <- gmB$x
## estimate Skeleton
skel.fm2 <- skeleton(suffStat = list(dm = X, adaptDF = FALSE),
                     indepTest = binCItest, alpha = 0.01,
                     labels = colnames(X), verbose = FALSE)

## show estimated Skeleton
par(mfrow = c(1,2))
plot(skel.fm2, main = "Estimated Skeleton")
plot(gmB$g, main = "True DAG")

In [ ]:
## Using simulated data
# Generating a DAG with node degrees >= 1
p <- 6
pconn <- 0.3
zerodegFlag <- TRUE

while (zerodegFlag ) {
  ## true DAG
  myDAG1 <- randomDAG(p, prob = pconn) 
  am <- (as(myDAG1,"matrix")!=0)*1L
  
  if (any(colSums(am + t(am)) == 0)) {
    zerodegFlag = TRUE
    next
  } else{
    zerodegFlag = FALSE
  }
}
plot(myDAG1, main = "randomDAG")

In [ ]:
# Generating 1000 observations from myDAG1 using standard normal error distribution
d.normMat10000 <- rmvDAG(10000, myDAG1, errDist="normal")
d.normMat1000 <- rmvDAG(1000, myDAG1, errDist="normal")
d.normMat500 <- rmvDAG(500, myDAG1, errDist="normal")


# Visualizing the distribution of the first four variables
par(mfrow=c(2,2))
for (i in 1:4){
  hist(d.normMat[,i])  
}

n <- nrow (d.normMat1000)
V <- colnames(d.normMat1000) # labels aka node names

skel.fit <- skeleton(suffStat = list(C = cor(d.normMat1000), n = n),
                     indepTest = gaussCItest, ## (partial correlations)
                     alpha = 0.01, labels = V, verbose = FALSE)
# show estimated Skeleton
par(mfrow=c(1,2))
plot(skel.fit, main = "Estimated Skeleton (1000)")
plot(myDAG1, main = "True DAG")

### PC Algorithm

In [ ]:
## PC
## Using Gaussian Data
n <- nrow (gmG8$ x)
V <- colnames(gmG8$ x) 
## estimate CPDAG
pc.fit <- pc(suffStat = list(C = cor(gmG8$x), n = n),
             indepTest = gaussCItest, 
             alpha=0.01, labels = V, verbose = FALSE)

## show estimated CPDAG
par(mfrow=c(1,2))
plot(pc.fit, main = "Estimated CPDAG")
plot(gmG8$g, main = "True DAG")



In [ ]:
## Using simulated data
n <- nrow (d.normMat500)
V <- colnames(d.normMat500) 

pc.fit500 <- pc(suffStat = list(C = cor(d.normMat500), n = n),
             indepTest = gaussCItest, 
             alpha=0.01, labels = V, verbose = FALSE)

n <- nrow (d.normMat10000)
V <- colnames(d.normMat1000) 

pc.fit10000 <- pc(suffStat = list(C = cor(d.normMat10000), n = n),
             indepTest = gaussCItest, 
             alpha=0.01, labels = V, verbose = FALSE)

# show estimated CPDAG
par(mfrow=c(1,3))
plot(myDAG1, main = "True DAG")
plot(pc.fit500, main = "CPDAG(500)")
plot(pc.fit10000, main = "CPDAG(10,000)")


### FCI and RFCI

In [ ]:
## FCI
data("gmL")

# visualizing the first 6 rows in the data set
# dimension of the data set (rows=observations, cols=Variables)
head(gmL$x)

# Number of nodes and edges in the true Model
gmL$g

suffStat1 <- list(C = cor(gmL$x), n = nrow(gmL$x))
pag.est <- fci(suffStat1, indepTest = gaussCItest, 
               p = ncol(gmL$x), alpha = 0.01, 
               labels = as.character(2:5))

par(mfrow = c(1, 2))
plot(pag.est)
mtext("FCI",side=3)
plot(gmL$g, main = "True Model")


In [ ]:
## RFCI
# same data set gML

suffStat1 <- list(C = cor(gmL$x), n = nrow(gmL$x))

pagrfci.est <- rfci(suffStat1, indepTest = gaussCItest, 
                p = ncol(gmL$x),alpha = 0.01, 
                labels = as.character(2:5))

par(mfrow = c(1, 3))
plot(pagfci.est)
mtext("FCI",side=3)
plot(pagrfci.est)
mtext('RFCI',side=3)
plot(gmL$g, main = "True Model")


### Comparing graphs via Structural Hamming Distance

In [ ]:
## Creating a graph from an adjacency matrix
Am <- rbind(c(0,1,0,0,1),
           c(0,0,0,1,1),
           c(1,0,0,1,0),
           c(1,0,0,0,1),
           c(0,0,0,0,0))

# shd() function needs the graphs as a graphNEL object
A <- as(getGraph(Am),"graphNEL")

# We remove the 3 -> 4 link in A and create the graph A1
A1 <- removeEdge(from="3",to="4",A)

# We remove the 4 -> 1 link in A1 and create the graph A2
A2 <- removeEdge(from="4",to="1",A1)


par(mfrow =c(1,3))
plot(A, main="A")
plot(A1, main="A1")
plot(A2, main="A2")

# computing and printing the distance values
shdAA <- shd(A,A)
shdAA1 <- shd(A,A1)
shdAA2 <- shd(A,A2)

print(shdAA)
print(shdAA1)
print(shdAA2)

In [ ]:
sessionInfo()